In [29]:
from mitools import reg
import numpy as np
import pandas as pd
from pandas_datareader.data import DataReader
from datetime import datetime
from statsmodels.tsa.regime_switching.tests.test_markov_regression import fedfunds, ogap, inf, areturns
import matplotlib.pyplot as plt

# Markov switching dynamic regression models from Statsmodels

https://www.statsmodels.org/stable/examples/notebooks/generated/markov_regression.html

This notebook provides an example of the use of Markov switching models in statsmodels to estimate dynamic regression models with changes in regime. It follows the examples in the Stata Markov switching documentation, which can be found at http://www.stata.com/manuals14/tsmswitch.pdf.

In [2]:
usrec = DataReader(
    "USREC", "fred", start=datetime(1947, 1, 1), end=datetime(2013, 4, 1)
)

## Federal funds rate with switching intercept

The first example models the federal funds rate as noise around a constant intercept, but where the intercept changes during different regimes. The model is simply: $$r_{t} = \mu_{S_t} + \varepsilon_{t}, \varepsilon_{t} \sim N(0, \sigma^2)$$

where $S_{t}\epsilon\left\{0,1 \right\}$, and the regime transitions according to $$P\left(S_{t} = s_{t} \mid S_{t-1} = s_{t-1} \right) = \begin{bmatrix}p_{00} & p_{10} \\1-p_{00} &  1 - p_{10} \\\end{bmatrix}$$

We will estimate the parameters of this model by maximum likelihood: $p_00, p_10, \mu_0, \mu_1, \sigma^2$.

The data used in this example can be found at https://www.stata-press.com/data/r14/usmacro.

In [ ]:
dta_fedfunds = pd.Series(
    fedfunds, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS"), name="fedfunds"
).to_frame()
_ = dta_fedfunds.plot(title="Federal funds rate", figsize=(12, 3))

In [4]:
mod_fedfunds = reg.MarkovRegressionModel(data=dta_fedfunds, dependent_variable="fedfunds", k_regimes=2)
res_fedfunds = mod_fedfunds.fit()

In [ ]:
res_fedfunds.summary()

From the summary output, the mean federal funds rate in the first regime (the “low regime”) is estimated to be $3.7$ whereas in the “high regime” it is $9.6$. Below we plot the smoothed probabilities of being in the high regime. The model suggests that the 1980’s was a time-period in which a high federal funds rate existed.

In [ ]:
pd.DataFrame(res_fedfunds.smoothed_marginal_probabilities)[1].plot(
    title="Probability of being in the high regime", figsize=(12, 3)
)

From the estimated transition matrix we can calculate the expected duration of a low regime versus a high regime.

In [ ]:
print(res_fedfunds.expected_durations)

A low regime is expected to persist for about fourteen years, whereas the high regime is expected to persist for only about five years.

### Federal funds rate with switching intercept and lagge dependent variable

The second example augments the previous model to include the lagged value of the federal funds rate.

$$r_{t} = \mu_{S_t} + \beta_{S_t} r_{t-1} + \varepsilon_{t}, \varepsilon_{t} \sim N(0, \sigma^2)$$
where $S_{t}\epsilon\left\{0,1 \right\}$, and the regime transitions according to $$P\left(S_{t} = s_{t} \mid S_{t-1} = s_{t-1} \right) = \begin{bmatrix}p_{00} & p_{10} \\1-p_{00} &  1 - p_{10} \\\end{bmatrix}$$
We will estimate the parameters of this model by maximum likelihood: 
$p_{00}, p_{10}, \mu_0, \mu_1,\beta_0, \beta_1, \sigma^2$.

In [8]:
dta_fedfunds['lfedfunds'] = dta_fedfunds['fedfunds'].shift(1)
mod_fedfunds2 = reg.MarkovRegressionModel(data=dta_fedfunds.iloc[1:], 
                                          dependent_variable="fedfunds", 
                                          k_regimes=2, 
                                          independent_variables=["lfedfunds"])
res_fedfunds2 = mod_fedfunds2.fit()

In [ ]:
res_fedfunds2.summary()

There are several things to notice from the summary output:

1. The information criteria have decreased substantially, indicating that this model has a better fit than the previous model.

2. The interpretation of the regimes, in terms of the intercept, have switched. Now the first regime has the higher intercept and the second regime has a lower intercept.

Examining the smoothed probabilities of the high regime state, we now see quite a bit more variability.

In [ ]:
_ = pd.DataFrame(res_fedfunds2.smoothed_marginal_probabilities)[0].plot(
    title="Probability of being in the high regime", figsize=(12, 3)
)

Finally, the expected durations of each regime have decreased quite a bit.

In [ ]:
print(res_fedfunds2.expected_durations)

### Taylor rule with 2 or 3 regimes

We now include two additional exogenous variables - a measure of the output gap and a measure of inflation - to estimate a switching Taylor-type rule with both 2 and 3 regimes to see which fits the data better.

Because the models can be often difficult to estimate, for the 3-regime model we employ a search over starting parameters to improve results, specifying 20 random search repetitions.

In [15]:
dta_ogap = pd.Series(ogap, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS"), name="ogap")
dta_inf = pd.Series(inf, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS"), name='inf')

dta_fedfunds_ogap_inf = pd.concat((dta_fedfunds, dta_ogap, dta_inf), axis=1).iloc[4:]

In [ ]:
dta_fedfunds_ogap_inf

In [21]:
mod_fedfunds3 = reg.MarkovRegressionModel(data=dta_fedfunds_ogap_inf, 
                                          dependent_variable="fedfunds", 
                                          k_regimes=2, 
                                          independent_variables=['lfedfunds', 'ogap', 'inf'])
res_fedfunds3 = mod_fedfunds3.fit()

In [24]:
np.random.seed(12345)
mod_fedfunds4 = reg.MarkovRegressionModel(data=dta_fedfunds_ogap_inf, 
                                          dependent_variable="fedfunds", 
                                          k_regimes=3, 
                                          independent_variables=['lfedfunds', 'ogap', 'inf'])
res_fedfunds4 = mod_fedfunds4.fit(search_reps=20)

In [ ]:
res_fedfunds3.summary()

In [ ]:
res_fedfunds4.summary()

Due to lower information criteria, we might prefer the 3-state model, with an interpretation of low-, medium-, and high-interest rate regimes. The smoothed probabilities of each regime are plotted below.

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7))

ax = axes[0]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[:, 2])
ax.set(title="Smoothed probability of a low-interest rate regime")

ax = axes[1]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[:, 0])
ax.set(title="Smoothed probability of a medium-interest rate regime")

ax = axes[2]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[:, 1])
ax.set(title="Smoothed probability of a high-interest rate regime")

fig.tight_layout()

### Switching variances

We can also accomodate switching variances. In particular, we consider the model $$y_{t} = \mu_{S_{t}} + y_{t-1}\beta_{S_t} + \varepsilon_t, \varepsilon_t \sim N(0, \sigma^2_{S_t})$$ We use maximum likelihood to estimate the parameters of this model: $$p_{00}, p_{10}, \mu_0, \mu_1, \beta_0, \beta_1, \sigma^2_0, \sigma^2_1$$

The application is to absolute returns on stocks, where the data can be found at https://www.stata-press.com/data/r14/snp500.

In [ ]:
dta_areturns = pd.Series(
    areturns, index=pd.date_range("2004-05-04", "2014-5-03", freq="W")
, name="areturns").to_frame()
_ = dta_areturns.plot(title="Absolute returns, S&P500", figsize=(12, 3))

In [37]:
dta_areturns["lareturns"] = dta_areturns["areturns"].shift(1)
mod_areturns = reg.MarkovRegressionModel(data=dta_areturns.iloc[1:], 
                                         dependent_variable="areturns", 
                                         k_regimes=2, 
                                         independent_variables=["lareturns"],
                                         switching_variance=True)
res_areturns = mod_areturns.fit()

In [ ]:
res_areturns.summary()

The first regime is a low-variance regime and the second regime is a high-variance regime. Below we plot the probabilities of being in the low-variance regime. Between 2008 and 2012 there does not appear to be a clear indication of one regime guiding the economy.

In [ ]:
_ = pd.DataFrame(res_areturns.smoothed_marginal_probabilities)[0].plot(
    title="Probability of being in a low-variance regime", figsize=(12, 3)
)

***